In [1]:
import geopandas as gpd 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
# la.explode(index_parts=False)
import random
from shapely.geometry import Polygon,Point

In [52]:
# access token
api_key = "R6Hv0RU5drMtW39Ci45p9ytQCsQSHJq7NXSMK0tPuMfhI5GkfgKNRkbjszp8"
import requests
import json

# remove warning message
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def call_api(endpoint, api_key, args, get=True):
    """Using user specified inputs, returns data from OpenET Raster API.

    Args:
        endpoint (str): Raster API endpoint

        api_key (str): Required api access key

        args (dictionary): User specified arguments for api call

    Returns:
        result (object): An object of Raster API results
    """

    # api server address
    server = 'https://openet-api.org/'

    # initialize request url
    url = server + endpoint

    # create header
    header = {"Authorization": api_key}

    if get:
        # make api get request
        resp = requests.get(url=url, headers=header, params=args, verify=False)

    else: 
        # make api post request
        resp = requests.post(url=url, headers=header, data=json.dumps(args), verify=False)

    # view results
    return resp.text

In [53]:
def open_et_call(day1,day2,lon,lat):
    # geodatbase api endpoint
  endpoint = "/raster/timeseries/point"

  # endpoint arguments
  args = {
    "date_range": [
      day1,day2
    ],
    "interval": "daily",
    "geometry": [lon,lat],
    "model": "ensemble",
    "variable": "et",
    "reference_et": "gridMET",
    "reducer": "mean",
    "units": "mm",
    "file_format": "JSON"
  }
  # query result
  response = call_api(endpoint, api_key, args, get=False)

  # cast into a dictionary
  response = json.loads(response)
  print(response)
  return pd.DataFrame.from_dict(response)


In [54]:
from datetime import datetime, timedelta
from collections import OrderedDict
from calendar import monthrange
def split_years_by_day(start_date, end_date):
    """
    Function to split query period into smaller periods with a difference of one day between start and end dates
    Takes input as start and end dates in the format 'YYYY-MM-DD'
    Returns lists of start and end periods for each day
    """

    start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in [start_date, end_date]]
    start = [(start + timedelta(_)).strftime(r"%Y-%m-%d") for _ in range((end - start).days)]
    end = [(datetime.strptime(item, "%Y-%m-%d") + timedelta(days=1)).strftime(r"%Y-%m-%d") for item in start]
    return start, end



# Example usage
start_dates, end_dates = split_years_by_day("2021-02-20", "2021-03-05")
print("Start dates:")
print(start_dates)
print("\nEnd dates:")
print(end_dates)

Start dates:
['2021-02-20', '2021-02-21', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021-02-26', '2021-02-27', '2021-02-28', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04']

End dates:
['2021-02-21', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021-02-26', '2021-02-27', '2021-02-28', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05']


## Regualar Model 

In [41]:
tmp=pd.read_csv("D:\\Backup\\Rouhin_Lenovo\\US_project\\Untitled_Folder\\Data\\Volk_merged\\ML_datasets\\Val_Data_for_OpenET\\US-xSB.csv",parse_dates=["Date"])
print(tmp.head())
tmp=tmp[tmp.Date.dt.year>=2016]
a,b=split_years_by_day(tmp["Date"].iloc[0].date().strftime('%Y-%m-%d'),(tmp["Date"].iloc[0]+timedelta(days=1)).strftime('%Y-%m-%d'))
# tmp["Date"].iloc[0]+datetime.timedelta(days=10)
b
# t.
# tmp.columns

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Backup\\Rouhin_Lenovo\\US_project\\Untitled_Folder\\Data\\Volk_merged\\ML_datasets\\Val_Data_for_OpenET\\US-xSB.csv'

In [40]:


api_key = "R6Hv0RU5drMtW39Ci45p9ytQCsQSHJq7NXSMK0tPuMfhI5GkfgKNRkbjszp8"
def arange_call(df):
    master_df=pd.DataFrame()
    sub_api=[]
    for i in range(0,df.shape[0]):
        print("Site=" , df["Site_name"].iloc[0])
        start_date,end_date=split_years_by_day(tmp["Date"].iloc[i].date().strftime('%Y-%m-%d'),(tmp["Date"].iloc[i]+timedelta(days=1)).strftime('%Y-%m-%d'))
        try:
            sub_api.append(open_et_call(start_date[0],end_date[0],df["Lon_volk"].iloc[i],df["Lat_volk"].iloc[i]))
        except Exception as e:
            print(f"error opening some_file_{i}")
            continue
    master_df["Date"]=pd.concat(sub_api)["time"]
    master_df["ETa"]=pd.concat(sub_api)["et"]
    # master_df["Area_ws_"+str(i)]=area_df["area"].iloc[i]
    master_df.to_csv("D:\\Backup\\Rouhin_Lenovo\\US_project\\Untitled_Folder\\Data\\Volk_merged\\ML_datasets\\Open_ET\\"+df["Site_name"].iloc[0]+".csv")
    return master_df
arange_call(tmp)

Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_0
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_1
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_2
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_3
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_4
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_5
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_6

ValueError: No objects to concatenate

### For sen1 

In [55]:
import os
os.chdir("D:\\Backup\\Rouhin_Lenovo\\US_project\\Untitled_Folder\\Data\\Volk_merged\\ML_datasets\\Val_Data_for_OpenET\\Sen1")
file_list=os.listdir()
# tmp=pd.read_csv("D:\\Backup\\Rouhin_Lenovo\\US_project\\Untitled_Folder\\Data\\Volk_merged\\ML_datasets\\Val_Data_for_OpenET\\Sen1\\JPL1_JV114.csv",parse_dates=["Date"])
tmp=pd.read_csv(file_list[9],parse_dates=["Date"])

print(tmp.head())
tmp=tmp[tmp.Date.dt.year>=2016]
a,b=split_years_by_day(tmp["Date"].iloc[0].date().strftime('%Y-%m-%d'),(tmp["Date"].iloc[0]+timedelta(days=1)).strftime('%Y-%m-%d'))
# tmp["Date"].iloc[0]+datetime.timedelta(days=10)
b
# t.
# tmp.columns

   Unnamed: 0.1  Unnamed: 0  Unnamed: 0_x       Date         B         R  \
0             0          19          32.0 2015-04-17  0.031523  0.055640   
1             1          22          35.0 2015-06-20  0.019918  0.023492   
2             2          24          36.0 2015-07-22  0.024867  0.023135   
3             3          26          39.0 2015-09-24  0.015655  0.026682   
4             4          30          42.0 2016-01-14 -0.007225  0.022365   

         GR       NIR    SWIR_1    SWIR_2  ...    VH_green      VH_blue  \
0  0.052202  0.184505  0.187475  0.112372  ... -202.403586  -335.188300   
1  0.040130  0.286640  0.143530  0.058197  ... -269.463697  -542.918446   
2  0.041092  0.299538  0.150927  0.060067  ... -324.712962  -536.574541   
3  0.035428  0.217257  0.111245  0.052065  ... -355.096793  -803.589374   
4  0.018460  0.140037  0.096945  0.059298  ... -529.881007  1353.855142   

      VH_NIR    VH_SWIR1    VH_SWIR2    VH_NDVI    VH_NDWI    VH_LST  \
0 -57.266595  -56.35

['2016-01-15']

In [56]:
api_key = "R6Hv0RU5drMtW39Ci45p9ytQCsQSHJq7NXSMK0tPuMfhI5GkfgKNRkbjszp8"
def arange_call(df):
    master_df=pd.DataFrame()
    sub_api=[]
    for i in range(0,df.shape[0]):
        print("Site=" , df["Site_name"].iloc[0])
        start_date,end_date=split_years_by_day(tmp["Date"].iloc[i].date().strftime('%Y-%m-%d'),(tmp["Date"].iloc[i]+timedelta(days=1)).strftime('%Y-%m-%d'))
        try:
            sub_api.append(open_et_call(start_date[0],end_date[0],df["Lon_volk"].iloc[i],df["Lat_volk"].iloc[i]))
        except Exception as e:
            print(f"error opening some_file_{i}")
            continue
    master_df["Date"]=pd.concat(sub_api)["time"]
    master_df["ETa"]=pd.concat(sub_api)["et"]
    # master_df["Area_ws_"+str(i)]=area_df["area"].iloc[i]
    master_df.to_csv("D:\\Backup\\Rouhin_Lenovo\\US_project\\Untitled_Folder\\Data\\Volk_merged\\ML_datasets\\Open_ET\\Sen1\\"+df["Site_name"].iloc[0]+".csv")
    return master_df
arange_call(tmp)

Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_0
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_1
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_2
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_3
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_4
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_5
Site= US-MOz
{'detail': 'Monthly request limit has been reached. Please wait until the 1st midnight UTC for reset.'}
error opening some_file_6

ValueError: No objects to concatenate